<a href="https://colab.research.google.com/github/rjpuskar/Unicorn-B-Gone/blob/main/Unicorn_B_Gone_BETA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instructions:

1. Export the raw text file from Unicorn
2. Open the text file, ctrl+a --> ctrl+c
3. Open excel, paste into cell A1
4. Save excel doc, note filename
5. Run first code box (press play icon in upper left corner), upload file

  Note: if this doesn't work, try a different browser

In [ ]:
from google.colab import files
uploaded = files.upload()

6. Change "filename" variable to match your filename
7. Run second code box (maybe run twice to force things to update)
8. If peak detection is desired, run third code box

In [ ]:
%reset-f

# change the following variable to equal your_filename.xlsx in quotes
filename = "Chrom_Data_Sample.xlsx"

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# reads copy/pasted excel file into dataframe
rawexcel = pd.read_excel(filename, header = 2)

# reads and displays injection point based on placement of "Injection" column
try:
    injcolumn = int(rawexcel.columns.get_loc("Injection"))
    injection = rawexcel.iat[0,(injcolumn - 1)]
    print("injection = ", injection)
except:
    print("Warning: no injection found")

# reads and displays list of fraction/waste start points based on placement of "Fraction" column
try:
  fraccolumn = int(rawexcel.columns.get_loc("Fraction"))
  rawexcel.loc[rawexcel.Fraction == "Waste", "Fraction"] = 0    # sets "Waste" values to 0 in "Fraction" column
  no_waste = rawexcel[rawexcel['Fraction'] > 0]
  no_waste_volume = no_waste.iloc[:,(fraccolumn-1)]
  fractions = no_waste_volume.to_list()
  print("fractions:", fractions)
  wastefrac = rawexcel[rawexcel["Fraction"] == 0]
  wastefrac_volume = wastefrac.iloc[:, (fraccolumn-1)]
  wastefractions = wastefrac_volume.tolist()
  print("waste:", wastefractions)
except:
    print("Warning: no fractions found")

## enable to show first 20 rows of rawexcel dataframe (for verification)
# rawexcel.head(21)

# calls for user-input elements to plot
print("max volume to display? (do not enter units, ex. 26)")
xrightlim = int(input()) # x axis maximum
print("minimum absorbance to display? (do not enter units, ex. 0)")
ybottomlim = int(input()) # y axis minimum
print("primary wavelength? (include units, ex. 280 nm)")
wavelength1 = input() # wavelength

# set font sizes
small_font = 14
medium_font = 16

plt.rc('font', size=small_font)          # default text sizes
plt.rc('axes', titlesize=small_font)     # axes title
plt.rc('axes', labelsize=medium_font)    # x and y labels
plt.rc('xtick', labelsize=small_font)    # tick labels
plt.rc('ytick', labelsize=small_font)    # tick labels
plt.rc('legend', fontsize=small_font)    # legend

# sets tick size
plt.rcParams["xtick.major.size"] = 10
plt.rcParams["xtick.major.width"] = 3
plt.rcParams["xtick.minor.size"] = 5
plt.rcParams["ytick.major.size"] = 10
plt.rcParams["ytick.major.width"] = 3
plt.rcParams["ytick.minor.size"] = 0

rawexcel.plot(kind="line", linewidth=2.5, x="ml", y="mAU", figsize=[12,8])

# axis options
plt.minorticks_on()
plt.xlim(right = xrightlim)
plt.ylim(bottom = ybottomlim)

plt.xlabel("Volume (mL)")
plt.ylabel("Absorbance (mAU)")

# plots vertical lines for injection (green), fractions (red), and waste (gray) using axis coordinates
try:
  plt.axvline(x=injection, linewidth=3, ymin=0, ymax=0.25, label="Injection", color="green")
except:
  print("error: no injection!")

try:
  for i in fractions:
    plt.axvline(x=i, linewidth=2, linestyle="dotted", ymin=0, ymax=0.25, color="red")
except:
  print("error: no fractions!")

try:
  for i in wastefractions:
    plt.axvline(x=i, linewidth=2, linestyle="dotted", ymin=0, ymax=0.25, color="gray")
except:
  print("error: no waste fractions!")

# sets legend options
legendlabel1 = plt.legend(loc="upper right")
legendlabel1.get_texts()[0].set_text(wavelength1)
plt.rcParams["legend.frameon"] = "False"

In [ ]:
from scipy.signal import find_peaks

firstabsonly = np.ndarray.flatten(rawexcel.iloc[:,1:2].values)

plt.figure(figsize=(10, 6), dpi=80)
peak, _ = find_peaks(firstabsonly, height=5, prominence=1.5) # finds the peaks, sets minimum height cutoff, higher prominence = less sensitive
plt.plot(firstabsonly) # plots absorbance only - currently assumes second column
plt.plot(peak, firstabsonly[peak], "*") # plots detected peaks with orange *
plt.plot(np.zeros_like(firstabsonly), "--", color="orange") # plots zero line
plt.show()

print("Peak absorbance values:")
print(firstabsonly[peak])